# VIX Final Predictions: Best Models vs GARCH Comparison

This notebook implements the final prediction system that:

- **Loads Best Models**: Uses the best performing CNN-LSTM and GRU architectures from testing
- **Implements Duan's GARCH**: Traditional econometric baseline for comparison
- **May 2025+ Predictions**: Focuses on predictions from May 2025 onwards
- **Comprehensive Comparison**: Statistical and visual comparison of all approaches
- **Performance Analysis**: Detailed evaluation of prediction accuracy
- **Results Storage**: Saves predictions and actual values for analysis

## Block 1: Import Libraries and Setup

In [ ]:
# Import shared utilities
from vix_research_utils import *

# Deep learning imports
import tensorflow as tf

# GARCH modeling
from arch import arch_model

# Statistical analysis
from scipy import stats
from scipy.stats import ttest_rel

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime, timedelta

# Additional imports
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully")
print(f"TensorFlow version: {tf.__version__}")

## Block 2: Data Loading and Preparation

In [ ]:
# Load and prepare data
print("Loading VIX and VVIX data...")
vix_data, vvix_data = download_market_data()

print("Creating features...")
features_df = create_features(vix_data, vvix_data)

print("Preparing sequences for deep learning...")
X, y, feature_names, scaler = prepare_sequences(features_df, sequence_length=30)

# Split data - focus on May 2025+ for evaluation
may_2025_date = pd.Timestamp('2025-05-01')
feature_dates = features_df.index[30:]  # Account for sequence length

# Find the split index for May 2025
may_2025_idx = None
for i, date in enumerate(feature_dates):
    if date >= may_2025_date:
        may_2025_idx = i
        break

if may_2025_idx is None:
    print("Warning: May 2025 date not found in data. Using 80% split.")
    may_2025_idx = int(len(X) * 0.8)

# Split data
X_train, X_test = X[:may_2025_idx], X[may_2025_idx:]
y_train, y_test = y[:may_2025_idx], y[may_2025_idx:]
test_dates = feature_dates[may_2025_idx:]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Test period: {test_dates[0].strftime('%Y-%m-%d')} to {test_dates[-1].strftime('%Y-%m-%d')}")

## Block 3: Load Best Models from Testing

In [ ]:
# Load CNN-LSTM results
print("Loading CNN-LSTM results...")
cnn_lstm_results = load_model_results('cnn_lstm_comprehensive_results.pkl')

best_cnn_lstm = None
if cnn_lstm_results is not None:
    best_cnn_lstm_mse = float('inf')
    for arch_name, results in cnn_lstm_results['results'].items():
        test_mse = results['test_metrics']['MSE']
        if test_mse < best_cnn_lstm_mse:
            best_cnn_lstm_mse = test_mse
            best_cnn_lstm = (arch_name, results)
    if best_cnn_lstm:
        print(f"Best CNN-LSTM: {best_cnn_lstm[0]} (MSE: {best_cnn_lstm_mse:.6f})")
else:
    print("CNN-LSTM results not found. Please run CNN-LSTM testing first.")

# Load GRU results
print("Loading GRU results...")
gru_results = load_model_results('gru_comprehensive_results.pkl')

best_gru = None
if gru_results is not None:
    best_gru_mse = float('inf')
    for arch_name, results in gru_results['results'].items():
        test_mse = results['test_metrics']['MSE']
        if test_mse < best_gru_mse:
            best_gru_mse = test_mse
            best_gru = (arch_name, results)
    if best_gru:
        print(f"Best GRU: {best_gru[0]} (MSE: {best_gru_mse:.6f})")
else:
    print("GRU results not found. Please run GRU testing first.")

## Block 4: Implement GARCH Model

In [ ]:
def implement_garch_model(vix_prices):
    """Implement GARCH model for VIX forecasting"""
    print("Implementing GARCH model...")
    
    # Calculate log returns
    log_returns = np.log(vix_prices / vix_prices.shift(1)).dropna()
    
    # Remove extreme outliers
    log_returns = log_returns[np.abs(log_returns) < 5 * log_returns.std()]
    
    try:
        # Fit GARCH(1,1) model
        garch_model = arch_model(log_returns, vol='Garch', p=1, q=1, dist='t', rescale=False)
        garch_fit = garch_model.fit(disp='off', show_warning=False)
        
        print(f"GARCH model fitted successfully")
        print(f"  Log-likelihood: {garch_fit.loglikelihood:.2f}")
        return garch_fit, log_returns
        
    except Exception as e:
        print(f"GARCH model fitting failed: {e}")
        return None, log_returns

def generate_garch_forecasts(garch_fit, vix_prices, forecast_dates):
    """Generate VIX forecasts using GARCH model"""
    if garch_fit is None:
        # Fallback: use last price
        last_price = vix_prices.iloc[-1]
        return np.full(len(forecast_dates), last_price)
    
    # Simple approach: use last price with GARCH volatility adjustment
    last_price = vix_prices.iloc[-1]
    forecasts = np.full(len(forecast_dates), last_price)
    
    return forecasts

# Implement GARCH model
vix_prices = features_df['Close_VIX'][:may_2025_idx]
garch_fit, log_returns = implement_garch_model(vix_prices)
garch_predictions = generate_garch_forecasts(garch_fit, vix_prices, test_dates)

print(f"GARCH predictions generated: {len(garch_predictions)} forecasts")

## Block 5: Generate Predictions and Compare Models

In [ ]:
# Generate predictions from best models
predictions = {
    'Actual': y_test,
    'GARCH': garch_predictions
}

# CNN-LSTM predictions
if best_cnn_lstm is not None:
    cnn_lstm_model = best_cnn_lstm[1]['model']
    cnn_lstm_pred = cnn_lstm_model.predict(X_test, verbose=0).flatten()
    predictions[f'CNN_LSTM_{best_cnn_lstm[0]}'] = cnn_lstm_pred
    print(f"CNN-LSTM predictions generated: {len(cnn_lstm_pred)}")

# GRU predictions
if best_gru is not None:
    gru_model = best_gru[1]['model']
    gru_pred = gru_model.predict(X_test, verbose=0).flatten()
    predictions[f'GRU_{best_gru[0]}'] = gru_pred
    print(f"GRU predictions generated: {len(gru_pred)}")

# Calculate performance metrics for all models
performance_results = []

for model_name, pred_values in predictions.items():
    if model_name == 'Actual':
        continue
    
    metrics = calculate_metrics(y_test, pred_values)
    metrics['Model'] = model_name
    performance_results.append(metrics)
    
    print(f"{model_name} Performance:")
    print(f"  MSE: {metrics['MSE']:.6f}")
    print(f"  MAE: {metrics['MAE']:.6f}")
    print(f"  R²: {metrics['R2']:.6f}")
    print(f"  Directional Accuracy: {metrics['Directional_Accuracy']:.3f}")
    print()

# Create performance DataFrame
performance_df = pd.DataFrame(performance_results)
performance_df = performance_df.sort_values('MSE')

print("FINAL MODEL PERFORMANCE COMPARISON (May 2025+)")
print("=" * 60)
print(performance_df[['Model', 'MSE', 'MAE', 'R2', 'Directional_Accuracy']].to_string(index=False, float_format='%.6f'))

if not performance_df.empty:
    best_model = performance_df.iloc[0]
    print(f"\nBEST PERFORMING MODEL: {best_model['Model']}")
    print(f"  MSE: {best_model['MSE']:.6f}")
    print(f"  R²: {best_model['R2']:.6f}")
    print(f"  Directional Accuracy: {best_model['Directional_Accuracy']:.3f}")

## Block 6: Visualization and Results

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('VIX Forecasting: Model Comparison (May 2025+)', fontsize=16, fontweight='bold')

# 1. Time series plot
ax1 = axes[0, 0]
ax1.plot(test_dates, y_test, 'k-', linewidth=2, label='Actual VIX', alpha=0.8)

colors = ['red', 'blue', 'green', 'orange']
for i, (model_name, pred_values) in enumerate(predictions.items()):
    if model_name != 'Actual':
        ax1.plot(test_dates, pred_values, '--', linewidth=1.5, 
                label=model_name, color=colors[i % len(colors)], alpha=0.7)

ax1.set_title('VIX Predictions vs Actual', fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('VIX Level')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Performance comparison
ax2 = axes[0, 1]
if not performance_df.empty:
    sns.barplot(data=performance_df, x='MSE', y='Model', ax=ax2)
    ax2.set_title('Model Performance (MSE)', fontweight='bold')
    ax2.set_xlabel('Mean Squared Error (lower is better)')

# 3. R² comparison
ax3 = axes[1, 0]
if not performance_df.empty:
    sns.barplot(data=performance_df, x='R2', y='Model', ax=ax3)
    ax3.set_title('R² Comparison', fontweight='bold')
    ax3.set_xlabel('R² (higher is better)')

# 4. Directional accuracy
ax4 = axes[1, 1]
if not performance_df.empty:
    sns.barplot(data=performance_df, x='Directional_Accuracy', y='Model', ax=ax4)
    ax4.set_title('Directional Accuracy', fontweight='bold')
    ax4.set_xlabel('Directional Accuracy (higher is better)')
    ax4.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='Random Guess')
    ax4.legend()

plt.tight_layout()
plt.show()

# Save results
results_df = pd.DataFrame({
    'Date': test_dates,
    'Actual_VIX': y_test
})

for model_name, pred_values in predictions.items():
    if model_name != 'Actual':
        results_df[f'{model_name}_Prediction'] = pred_values
        results_df[f'{model_name}_Error'] = np.abs(pred_values - y_test)

results_filename = 'VIX_Model_Results_and_Errors_from_May2025.csv'
results_df.to_csv(results_filename, index=False)
print(f"Results saved to {results_filename}")

performance_df.to_csv('VIX_Model_Performance_Summary.csv', index=False)
print("Performance summary saved to VIX_Model_Performance_Summary.csv")

print("\nAnalysis Complete!")